In [7]:
import duckdb
import numpy as np
import orjson
import datetime as dt
from pprint import pprint
from typing import Any, Dict, Hashable, Optional, List
import tqdm

# Sklearn Classification Metrics with DuckDB

This notebook demonstrates **DuckDB** for Delta Lake access instead of Polars.

## Migration: Polars → DuckDB

| Aspect | Polars | DuckDB |
|--------|--------|--------|
| **Connection** | `pl.scan_delta(path, storage_options={...})` | `conn.execute(delta_scan(path))` |
| **SQL Context** | `pl.SQLContext().register("data", lf)` | Direct SQL via `delta_scan()` |
| **Query → DataFrame** | `sql.execute("SELECT...").collect()` | `conn.execute("SELECT...").df()` |
| **Query → NumPy** | `result.to_numpy()` | `conn.execute("SELECT...").fetchnumpy()` |
| **To Dict** | `result.to_dicts()` | `result.to_dict('records')` |

## Benefits of DuckDB
- **Faster** for analytical queries on Delta Lake
- **Native SQL** interface (no DataFrame→SQL conversion)
- **Direct pandas output** via `.df()` method
- **Direct NumPy output** via `.fetchnumpy()` - skips pandas entirely!
- **Lower memory** footprint for large queries

## Optimal Pipeline for Sklearn

```
DuckDB delta_scan() → fetchnumpy() → np.column_stack() → Sklearn
```

This is **faster** than going through pandas because sklearn internally converts DataFrames to numpy anyway.

In [8]:
# =============================================================================
# MINIO/S3 CONFIGURATION
# =============================================================================
# Same credentials as Polars, but formatted for DuckDB's httpfs extension
MINIO_HOST = "localhost"
MINIO_PORT = "9000"
MINIO_ENDPOINT = f"{MINIO_HOST}:{MINIO_PORT}"
MINIO_ACCESS_KEY = "minioadmin"
MINIO_SECRET_KEY = "minioadmin123"
PROJECT_NAME = "Transaction Fraud Detection"

In [9]:
# =============================================================================
# DELTA LAKE PATHS
# =============================================================================
DELTA_PATHS = {
    "Transaction Fraud Detection": "s3://lakehouse/delta/transaction_fraud_detection",
    "Estimated Time of Arrival": "s3://lakehouse/delta/estimated_time_of_arrival",
    "E-Commerce Customer Interactions": "s3://lakehouse/delta/e_commerce_customer_interactions",
    "Sales Forecasting": "s3://lakehouse/delta/sales_forecasting",
}

delta_path = DELTA_PATHS.get(PROJECT_NAME)

In [12]:
# =============================================================================
# DUCKDB CONNECTION SETUP
# =============================================================================
# DuckDB Delta Lake connection - equivalent to Polars scan_delta()
# 
# Key differences from Polars:
# - Uses SQL-based delta_scan() function instead of scan_delta()
# - Requires httpfs extension for S3/MinIO access
# - Requires delta extension for Delta Lake format
# - Configuration via CREATE SECRET instead of SET statements
#
# IMPORTANT: We use CREATE SECRET to avoid DuckDB trying to fetch credentials
# from AWS EC2 metadata service (169.254.169.254)
# =============================================================================
import os

# Disable AWS EC2 metadata service lookup (prevents 169.254.169.254 errors)
os.environ["AWS_EC2_METADATA_DISABLED"] = "true"

# Create connection (in-memory database)
conn = duckdb.connect()

# Install and load required extensions
conn.execute("INSTALL delta; LOAD delta;")
conn.execute("INSTALL httpfs; LOAD httpfs;")

# Create a secret for S3/MinIO credentials
# This is the recommended way to configure S3 access in DuckDB
# and avoids the EC2 metadata service lookup issue
conn.execute(f"""
    CREATE SECRET minio_secret (
        TYPE S3,
        KEY_ID '{MINIO_ACCESS_KEY}',
        SECRET '{MINIO_SECRET_KEY}',
        REGION 'us-east-1',
        ENDPOINT '{MINIO_ENDPOINT}',
        URL_STYLE 'path',
        USE_SSL false
    );
""")

print("DuckDB extensions loaded and S3 secret configured")

DuckDB extensions loaded and S3 secret configured


In [13]:
# =============================================================================
# QUERY DELTA LAKE TABLE
# =============================================================================
# DuckDB equivalent of Polars:
#   lf = pl.scan_delta(delta_path, storage_options=DELTA_STORAGE_OPTIONS)
#   sql = pl.SQLContext()
#   sql.register("data", lf)
#   result = sql.execute("SELECT * FROM data LIMIT 1000").collect()
#
# DuckDB uses delta_scan() function directly in SQL
# Returns pandas DataFrame via .df() method
# =============================================================================

# Query with delta_scan() - direct Delta Lake access
query = f"SELECT * FROM delta_scan('{delta_path}') LIMIT 1000"
result = conn.execute(query).df()

print(f"Loaded {len(result)} rows from Delta Lake")
print(f"Columns: {list(result.columns)}")
result.head()

Loaded 1000 rows from Delta Lake
Columns: ['transaction_id', 'user_id', 'timestamp', 'amount', 'currency', 'merchant_id', 'product_category', 'transaction_type', 'payment_method', 'location', 'ip_address', 'device_info', 'user_agent', 'account_age_days', 'cvv_provided', 'billing_address_match', 'is_fraud']


,transaction_id,user_id,timestamp,amount,currency,merchant_id,product_category,transaction_type,payment_method,location,ip_address,device_info,user_agent,account_age_days,cvv_provided,billing_address_match,is_fraud
0,9b055e08-70e5-4ee8-b65a-b14cfbd0656d,b321cf7b-783f-41f4-892b-7d5fa50b7b36,2026-01-16T14:07:08.076082+00:00,308.27,USD,merchant_43,clothing,withdrawal,crypto,"{""lat"":-28.486754,""lon"":40.419891}",190.120.132.26,"{""os"":""Android"",""browser"":""Other""}",Mozilla/5.0 (iPhone; CPU iPhone OS 15_8_2 like...,1133,True,True,0
1,cfa5ac39-577b-4e12-a3e4-3f6fe2f6e0f9,b4a44b10-b154-463e-b516-8c3dec4ad496,2026-01-16T14:07:08.234866+00:00,200.66,AUD,merchant_53,travel,purchase,debit_card,"{""lat"":-36.855021,""lon"":-28.310892}",79.240.163.54,"{""os"":""macOS"",""browser"":""Opera""}",Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6...,1592,True,True,0
2,97bdca8a-6b56-4ddf-b0da-bd6e46e08a8a,c6a4e1ca-9bca-4ef4-a212-0c4745df56ba,2026-01-16T14:07:08.496999+00:00,315.85,GBP,merchant_16,groceries,purchase,paypal,"{""lat"":-54.1930465,""lon"":-88.064682}",159.193.162.175,"{""os"":""Android"",""browser"":""Firefox""}",Opera/8.76.(X11; Linux x86_64; ar-ER) Presto/2...,1802,True,True,0
3,3adf9fc2-e285-4a2b-859e-c8b592255ffa,584e0e22-e441-4d85-86b9-0e91e9d82feb,2026-01-16T14:07:08.814717+00:00,171.71,JPY,merchant_12,clothing,purchase,debit_card,"{""lat"":34.7964225,""lon"":-88.678611}",171.108.91.197,"{""os"":""macOS"",""browser"":""Other""}",Mozilla/5.0 (iPad; CPU iPad OS 13_5_1 like Mac...,1824,True,True,0
4,bbd1d63f-dc47-4f4e-82f2-9ea2f35aed6b,608d5bce-b0cb-45b8-adb5-e881e54ae90b,2026-01-16T14:07:08.984244+00:00,123.66,AUD,merchant_39,gambling,withdrawal,debit_card,"{""lat"":22.479582,""lon"":-105.457153}",165.169.79.123,"{""os"":""Windows"",""browser"":""Firefox""}",Mozilla/5.0 (Macintosh; PPC Mac OS X 10_5_1; r...,219,True,True,0


In [14]:
# =============================================================================
# REUSABLE HELPER: DuckDB Delta Lake Query Function
# =============================================================================
# Encapsulates the DuckDB Delta Lake connection pattern for reuse
# =============================================================================

def query_delta_lake(
    delta_path: str,
    query: str = "SELECT * FROM data",
    minio_endpoint: str = "localhost:9000",
    minio_access_key: str = "minioadmin",
    minio_secret_key: str = "minioadmin123",
    use_ssl: bool = False,
) -> list:
    """
    Query a Delta Lake table using DuckDB and return list of dicts.
    
    This is the DuckDB equivalent of:
        lf = pl.scan_delta(delta_path, storage_options=DELTA_STORAGE_OPTIONS)
        sql = pl.SQLContext()
        sql.register("data", lf)
        result = sql.execute(query).collect()
        return result.to_dicts()
    
    Args:
        delta_path: S3 path to Delta Lake table (e.g., "s3://lakehouse/delta/table")
        query: SQL query to execute. Use 'data' as the table name.
        minio_endpoint: MinIO/S3 endpoint (host:port)
        minio_access_key: S3 access key
        minio_secret_key: S3 secret key
        use_ssl: Whether to use HTTPS (False for local MinIO)
    
    Returns:
        List of dictionaries (records)
    
    Example:
        samples = query_delta_lake(
            delta_path="s3://lakehouse/delta/transaction_fraud_detection",
            query="SELECT * FROM data WHERE is_fraud = 1 LIMIT 100"
        )
    """
    # Disable AWS EC2 metadata lookup
    os.environ["AWS_EC2_METADATA_DISABLED"] = "true"
    
    conn = duckdb.connect()
    
    # Load extensions
    conn.execute("INSTALL delta; LOAD delta;")
    conn.execute("INSTALL httpfs; LOAD httpfs;")
    
    # Configure S3 using CREATE SECRET (avoids EC2 metadata issues)
    conn.execute(f"""
        CREATE SECRET minio_secret (
            TYPE S3,
            KEY_ID '{minio_access_key}',
            SECRET '{minio_secret_key}',
            REGION 'us-east-1',
            ENDPOINT '{minio_endpoint}',
            URL_STYLE 'path',
            USE_SSL {'true' if use_ssl else 'false'}
        );
    """)
    
    # Replace 'data' table reference with delta_scan()
    actual_query = query.replace("FROM data", f"FROM delta_scan('{delta_path}')")
    actual_query = actual_query.replace("from data", f"FROM delta_scan('{delta_path}')")
    
    # Execute and return as list of dicts
    df = conn.execute(actual_query).df()
    conn.close()
    
    return df.to_dict('records')


# Test the helper function
test_samples = query_delta_lake(
    delta_path=delta_path,
    query="SELECT * FROM data LIMIT 5"
)
print(f"Helper function test: {len(test_samples)} rows retrieved")

Helper function test: 5 rows retrieved


In [15]:
# =============================================================================
# FETCHNUMPY: SKIP PANDAS, GO DIRECTLY TO NUMPY
# =============================================================================
# DuckDB's fetchnumpy() returns a dict of numpy arrays - one per column.
# This is FASTER than .df() because it skips pandas DataFrame creation.
#
# Performance comparison:
#   .df()         → DuckDB → pandas DataFrame → .values → numpy (2 conversions)
#   .fetchnumpy() → DuckDB → numpy arrays directly (0 conversions)
#
# Sklearn internally converts everything to numpy, so fetchnumpy() is optimal.
# =============================================================================

# Define features for Transaction Fraud Detection
TFD_NUMERICAL_FEATURES = [
    "amount",
    "account_age_days",
]

TFD_BOOLEAN_FEATURES = [
    "cvv_provided",
    "billing_address_match",
]

TFD_TARGET = "is_fraud"

# Build query with only the columns we need (more efficient)
selected_columns = TFD_NUMERICAL_FEATURES + TFD_BOOLEAN_FEATURES + [TFD_TARGET]
columns_sql = ", ".join(selected_columns)

query_numpy = f"SELECT {columns_sql} FROM delta_scan('{delta_path}') LIMIT 1000"
print(f"Query: {query_numpy}\n")

# fetchnumpy() returns dict[str, np.ndarray]
data_numpy = conn.execute(query_numpy).fetchnumpy()

print(f"Type: {type(data_numpy)}")
print(f"Keys: {list(data_numpy.keys())}")
print(f"\nArray shapes:")
for col, arr in data_numpy.items():
    print(f"  {col}: {arr.shape}, dtype={arr.dtype}")

Query: SELECT amount, account_age_days, cvv_provided, billing_address_match, is_fraud FROM delta_scan('s3://lakehouse/delta/transaction_fraud_detection') LIMIT 1000

Type: <class 'dict'>
Keys: ['amount', 'account_age_days', 'cvv_provided', 'billing_address_match', 'is_fraud']

Array shapes:
  amount: (1000,), dtype=float64
  account_age_days: (1000,), dtype=int32
  cvv_provided: (1000,), dtype=bool
  billing_address_match: (1000,), dtype=bool
  is_fraud: (1000,), dtype=int32


In [ ]:
# =============================================================================
# CONVERT FETCHNUMPY DICT TO SKLEARN X, y FORMAT
# =============================================================================
# Sklearn expects:
#   X: 2D array of shape (n_samples, n_features)
#   y: 1D array of shape (n_samples,)
#
# np.column_stack() combines 1D arrays into a 2D feature matrix
# =============================================================================

# Combine feature columns into X matrix
feature_columns = TFD_NUMERICAL_FEATURES + TFD_BOOLEAN_FEATURES
X = np.column_stack([data_numpy[col] for col in feature_columns])

# Extract target column as y
y = data_numpy[TFD_TARGET]

print(f"X shape: {X.shape}")  # (n_samples, n_features)
print(f"y shape: {y.shape}")  # (n_samples,)
print(f"X dtype: {X.dtype}")
print(f"y dtype: {y.dtype}")
print(f"\nFeature columns: {feature_columns}")
print(f"\nFirst 3 rows of X:\n{X[:3]}")
print(f"\nFirst 10 values of y: {y[:10]}")